In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader, Dataset
from PIL import Image

c:\Users\Genral Rady\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Notebook For bigger dataset with brown Backgrounds and white

In [2]:
# Create output directory if it doesn't exist
output_dir = r"D:\GAN\NewSec"
os.makedirs(output_dir, exist_ok=True)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_dim = 100
img_height, img_width, channels = 64, 64, 3
batch_size = 64
learning_rate = 0.0001
num_epochs = 40

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*channels, [0.5]*channels)
])

In [3]:
# Custom dataset loader for unlabeled images
class UnlabeledImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [os.path.join(root_dir, file) for file in os.listdir(root_dir) if file.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, 0  # Dummy label

# Dataset and DataLoader
dataset = UnlabeledImageDataset(r"D:\Second hand", transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [4]:
# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu = nn.LeakyReLU(0.2)
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(in_channels)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        out = self.relu(out)
        return out

# Generator
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.init_size = img_height // 16
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 512 * self.init_size ** 2))
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(512),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            ResidualBlock(64),
            ResidualBlock(64),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(64, channels, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 512, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

In [5]:
# Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(channels, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Flatten(),
            nn.Linear(512 * 4 * 4, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        validity = self.model(img)
        return validity

In [6]:
# Initialize generator and discriminator
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)
# Loss function
adversarial_loss = nn.BCELoss()

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))

In [7]:
# Function to save generated images
def save_generated_images(epoch, generator, latent_dim, output_dir):
    noise = torch.randn(16, latent_dim, device=device)
    generated_images = generator(noise).detach().cpu()
    generated_images = (generated_images + 1) / 2  # Rescale images to [0, 1]
    save_image(generated_images, os.path.join(output_dir, f"gan_generated_image_epoch_{epoch}.png"), nrow=4, normalize=True)

In [8]:
for epoch in range(num_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        real_images = imgs.to(device)

        # Adversarial ground truths
        valid = torch.ones(real_images.size(0), 1, device=device)
        fake = torch.zeros(real_images.size(0), 1, device=device)

        # -----------------
        #  Train Generator
        # -----------------
        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = torch.randn(real_images.size(0), latent_dim, device=device)

        # Generate a batch of images
        gen_images = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_images), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()

        # Loss for real images
        real_loss = adversarial_loss(discriminator(real_images), valid)
        # Loss for fake images
        fake_loss = adversarial_loss(discriminator(gen_images.detach()), fake)
        # Total discriminator loss
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        # Print losses
        print(f"[Epoch {epoch}/{num_epochs}] [Batch {i}/{len(dataloader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")

    # Save generated images
    save_generated_images(epoch, generator, latent_dim, output_dir)
    # Save the generator weights
torch.save(generator.state_dict(), os.path.join(output_dir, 'generator_final.pth'))

# Save the discriminator weights
torch.save(discriminator.state_dict(), os.path.join(output_dir, 'discriminator_final.pth'))

[Epoch 0/40] [Batch 0/83] [D loss: 0.7063225507736206] [G loss: 0.7303787469863892]
[Epoch 0/40] [Batch 1/83] [D loss: 0.48075002431869507] [G loss: 0.7692115902900696]
[Epoch 0/40] [Batch 2/83] [D loss: 0.3996565639972687] [G loss: 0.897209644317627]
[Epoch 0/40] [Batch 3/83] [D loss: 0.34309297800064087] [G loss: 1.1480565071105957]
[Epoch 0/40] [Batch 4/83] [D loss: 0.31014394760131836] [G loss: 1.1851757764816284]
[Epoch 0/40] [Batch 5/83] [D loss: 0.34137624502182007] [G loss: 1.2779899835586548]
[Epoch 0/40] [Batch 6/83] [D loss: 0.3643658757209778] [G loss: 1.3779096603393555]
[Epoch 0/40] [Batch 7/83] [D loss: 0.3095884323120117] [G loss: 1.4027963876724243]
[Epoch 0/40] [Batch 8/83] [D loss: 0.29804301261901855] [G loss: 1.7764554023742676]
[Epoch 0/40] [Batch 9/83] [D loss: 0.3984619379043579] [G loss: 1.5269798040390015]
[Epoch 0/40] [Batch 10/83] [D loss: 0.3864801228046417] [G loss: 1.386504888534546]
[Epoch 0/40] [Batch 11/83] [D loss: 0.46496710181236267] [G loss: 1.4419

In [10]:
import torch
from torchvision.utils import save_image
import os

# Assume the Generator and Discriminator classes have been defined somewhere

# Initialize the models
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)

# Load the weights from files
generator.load_state_dict(torch.load(r"D:\GAN\NewSec\4kgenerator_final.pth"))
discriminator.load_state_dict(torch.load(r"D:\GAN\NewSec\4kdiscriminator_final.pth"))

# Set to evaluation mode
generator.eval()
discriminator.eval()

# Specify the directory for saving images
output_dir = r"D:\GAN\NewSec"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Generate images
with torch.no_grad():  # Important for inference to disable gradient calculation
    z = torch.randn(16, latent_dim, device=device)  # 16 is an example batch size for generating 16 images
    generated_images = generator(z)
    image_path = os.path.join(output_dir, 'generated_images3.png')
    save_image(generated_images, image_path, nrow=4, normalize=True)

print(f"Generated images saved to '{image_path}'.")

Generated images saved to 'D:\GAN\NewSec\generated_images3.png'.
